In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import norm

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read excel file into dataframe concatenated with all sheets
def read_and_clean_data(filepath, df_name):
    # Read treatment data into dataframes
    no_diabetes = pd.read_excel(filepath, sheet_name='No_diabetes')
    pre_diabetes = pd.read_excel(filepath, sheet_name='Pre_diabetes')
    diabetes = pd.read_excel(filepath, sheet_name='Diabetes')

    # Create labels for each group
    no_diabetes['no_diabetes'] = 1
    no_diabetes['pre_diabetes'] = 0
    no_diabetes['diabetes'] = 0

    pre_diabetes['no_diabetes'] = 0
    pre_diabetes['pre_diabetes'] = 1
    pre_diabetes['diabetes'] = 0

    diabetes['no_diabetes'] = 0
    diabetes['pre_diabetes'] = 0
    diabetes['diabetes'] = 1

    # Concatenate dataframes
    treatment = pd.concat([no_diabetes, pre_diabetes, diabetes], ignore_index=True)
    
    # Drop the extra columns with error handling to avoid issues on rerun
    columns_to_drop = ['No diabetes', 'Pre-Diabetes', 'Diabetes']
    for col in columns_to_drop:
        try:
            treatment = treatment.drop(columns=[col])
        except:
            pass

    # Fill NaN values with 0 in specified columns
    columns_to_fill = ['Length of Stay', 'Fall', 'U69.*', 'T81.*', 'T84.*', 'N17.*', 'I21.*', 'R96.*', 'TOD', 'INTENSIV']
    treatment[columns_to_fill] = treatment[columns_to_fill].fillna(0)

    # Rename columns
    treatment = treatment.rename(columns={ 'U69.*': 'U69', 'T81.*': 'T81', 'T84.*': 'T84', 'N17.*': 'N17', 'I21.*': 'I21', 'R96.*': 'R96',})

    #Assigning the user defined name to the dataframe
    globals()[df_name] = treatment


In [ ]:
#read treatment group 1 data into dataframe
filepath = r'D:\OneDrive - National University of Sciences & Technology\Desktop\upwork\MedicalStats\Data\Raw\Treatment group 1.xlsx'
read_and_clean_data(filepath, 'treatment_1')
treatment_5.head()

In [ ]:
#read treatment group 3 data into dataframe
filepath = r'D:\OneDrive - National University of Sciences & Technology\Desktop\upwork\MedicalStats\Data\Raw\Treatment group 3.xlsx'
read_and_clean_data(filepath, 'treatment_3')
treatment_5.head()

In [ ]:
#read treatment group 5 data into dataframe
filepath = r'D:\OneDrive - National University of Sciences & Technology\Desktop\upwork\MedicalStats\Data\Raw\Treatment group 5.xlsx'
read_and_clean_data(filepath, 'treatment_5')
treatment_5.head()